# Perceptró aplicat a Olivetti

**Lectura del corpus:** $\;$ comprovem també que les matrius de dades i etiquetes tenes les files i columnes que toca, alabado sea jesus


In [10]:
import numpy as np; from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
olivetti = fetch_olivetti_faces(); X = olivetti.data.astype(np.float16); X/= X.max()
y = olivetti.target.astype(np.uint).reshape(-1, 1);
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, shuffle = True,random_state = 23)
print(X_train.shape, X_test.shape)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data
(320, 4096) (80, 4096)


In [12]:
def perceptro(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

In [ ]:
for b in(.0, .1, 1,10,100,1000,10000):
  W, E, k = perceptro(X_train, y_train, b=b, K=1000)
  X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
  y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
  err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
  print(f"b: {b} E: {E} k: {k} err_test: {err_test:.1%}")

b: 0.0 E: 0 k: 48 err_test: 18.8%
b: 0.1 E: 0 k: 42 err_test: 7.5%
b: 1 E: 0 k: 79 err_test: 11.2%
b: 10 E: 0 k: 45 err_test: 15.0%
b: 100 E: 0 k: 72 err_test: 8.8%


**Interpretació de resultats:** $\;$ les dades d'entrenament no semblen linealment separables; no està clar que un marge major que zero puga millorar resultats, sobretot perquè sols tenim $30$ mostres de test; amb marge nul ja hem vist que s'obté un error (en test) del $16.7\%$